In [1]:
# sys
import sys

# warning off
import warnings
# IO
import json

# logging
from loguru import logger

# set logger level
logger.remove(0)
logger.add(sys.stderr, level="INFO")

warnings.filterwarnings("ignore")

In [2]:
# reading annotation files
with open('dataset/annotations.json') as fp:
    raw_annotations = json.load(fp)

with open('dataset/annotated-osm-entities.json') as fp:
    second_annotator = json.load(fp)

In [15]:
first_annotator = {}
for annotation_record in raw_annotations:
    page = None
    annotation = []
    for info in annotation_record:
        if 'page' in info.keys():
            page = str(info['page'])
        elif 'type' in info.keys() and 'osm_id' in info.keys():
            annotation.append(f"{info['type']}:{info['osm_id']}")
        else:
            logger.error(f'structural issues: {info}')

    if page is not None:
        first_annotator[page] = annotation

logger.info(f'total annotations: {len(first_annotator)}')

2024-06-23 21:11:21.525 | INFO     | __main__:<module>:16 - total annotations: 664


In [16]:
first_annotator['6']

['way:69366108', 'way:69366081']

In [19]:
processed_second_annotator = {}
for key, values in second_annotator.items():
    new_values = []
    for v in values:
        if len(v) != 2:
            logger.info(f'structural issue in second annotator results: {key}: {v}')
        new_values.append(f"{v[0]}:{v[1]}")
    processed_second_annotator[key] = new_values

In [20]:
second_annotator['6']

[['way', '69366108'], ['node', '5315720235']]

In [21]:
processed_second_annotator['6']

['way:69366108', 'node:5315720235']

In [22]:
second_annotator = processed_second_annotator

In [23]:
list(set(second_annotator['6']).intersection(first_annotator['6']))

['way:69366108']

In [25]:
matched = []
not_matched_from_first = []
not_matched_from_second = []

# agreed_first_and_second =[]
first_found = []
second_found = []
total = []
for poi_id, values in second_annotator.items():
    if poi_id not in first_annotator.keys():
        logger.info(f'not annotated by first annotator: {poi_id}')
        continue
    total.append(poi_id)
    first_values = first_annotator[poi_id]
    if len(values) > 0:
        second_found.append(poi_id)
    if len(first_values) > 0:
        first_found.append(poi_id)
    shared_osm_ids = list(set(second_annotator[poi_id]).intersection(first_annotator[poi_id]))
    matched.extend(shared_osm_ids)
    not_matched_from_first.extend([f for f in first_values if f not in shared_osm_ids])
    not_matched_from_second.extend([s for s in values if s not in shared_osm_ids])

2024-06-23 21:14:15.091 | INFO     | __main__:<module>:11 - not annotated by first annotator: 20


In [27]:
shared_founds = list(set(second_found).intersection(first_found))
logger.info(f'total overlap: {len(total)} - first founds: {len(first_found)} - second founds: {len(second_found)} - shared: {len(shared_founds)}')

2024-06-23 21:15:53.907 | INFO     | __main__:<module>:2 - total overlap: 349 - first founds: 240 - second founds: 205 - shared: 187


In [ ]:
logger.info(f'matched: {len(matched)} - not matched from first: {len(not_matched_from_first)} - not matched from second: {len(not_matched_from_second)}')